In [5]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time
from selenium.webdriver.common.keys import Keys

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 음식점 리스트 파일 로드
with open('/mnt/data/200m아래_음식점리스트_혜정지연_합본.txt', 'r', encoding='utf-8') as file:
    store_names = [line.strip() for line in file.readlines()]

for store_name in store_names:
    try:
        # 네이버 지도 웹사이트로 이동
        driver.get('https://map.naver.com/v5/')
        time.sleep(3)  # 페이지 로드 대기

        # 검색창 요소가 나타날 때까지 대기 후 검색어 입력
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input.input_search"))
        )
        search_box.send_keys(store_name)
        search_box.send_keys(Keys.RETURN)

        # 검색 결과 로드 대기
        time.sleep(5)

        # 검색 결과 iframe으로 전환
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))

        # 첫 번째 검색 결과 클릭
        first_result_div = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]'))
        )
        first_result_div.click()
        time.sleep(3)  # 클릭 후 페이지가 로드되기를 기다림

        # 메인 컨텐츠로 다시 전환
        driver.switch_to.default_content()

        # entryIframe이 로드될 때까지 대기 후 전환
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
        time.sleep(3)

        # 가게 정보 가져오기
        store_name_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
        )
        store_name_text = store_name_element.text

        # 업종 카테고리 가져오기
        category_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
        )
        category = category_element.text

        # 별점 가져오기
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()

        # 방문자 리뷰 가져오기
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

        # 블로그 리뷰 가져오기
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()

        # 결과 출력
        print("가게 이름:", store_name_text)
        print("업종 카테고리:", category)
        print("별점:", rating)
        print("방문자 리뷰:", visitor_review)
        print("블로그 리뷰:", blog_review)

        # '리뷰' 탭 클릭하기
        try:
            review_tab_element = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
            )
            review_tab_element.click()
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
            print("리뷰 탭 클릭 완료")
        except Exception as e:
            print("리뷰 탭 클릭 오류:", e)

        # 리뷰 총 개수 가져오기
        print("\n리뷰 총 개수 가져오기")
        try:
            review_count_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
            )
            total_reviews = int(review_count_element.text.replace(',', ''))
            print("총 리뷰 수:", total_reviews)
        except Exception as e:
            print("리뷰 총 개수 가져오기 오류:", e)

        review_count = 0
        reviews = []
        review_text_set = set()  # 중복 확인을 위한 set

        while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
            review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
            total_elements_on_page = len(review_elements)

            for i in range(review_count + 1, total_elements_on_page + 1):
                retry_count = 0
                while retry_count < 3:  # 최대 3번 재시도
                    try:
                        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                        current_review_element = review_elements[i - 1]

                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                        time.sleep(0.1)

                        review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                        review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                        review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                        review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                        try:
                            review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                            review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                        except NoSuchElementException:
                            review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                            review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                        review_text = review_text_element.text.strip()
                        review_date = review_date_element.text.strip()

                        if review_text and review_text not in review_text_set:
                            reviews.append((review_date, review_text))
                            review_text_set.add(review_text)
                            review_count += 1
                            print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                        else:
                            print(f"중복된 리뷰 발견, 저장하지 않음: {review_text}")
                        break
                    except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                        retry_count += 1
                        print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                        time.sleep(0.5)

            try:
                bottom_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
                )
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
                driver.execute_script("arguments[0].click();", bottom_more_button)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
                )
                print("하단 '더보기' 버튼 클릭 완료")
            except (NoSuchElementException, TimeoutException):
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(1)
                print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")

        print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

        # CSV 파일로 저장
        reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
        reviews_df.index = range(1, len(reviews) + 1)
        csv_filename = f"{store_name_text}_reviews.csv"
        reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
        print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.\n")

    except Exception as e:
        print(f"{store_name}에서 오류 발생:", e)

# 드라이버 종료
driver.quit()

스노우폭스 뱅뱅점의 첫 번째 검색 결과를 클릭할 수 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x006EEBD3+24307]
	(No symbol) [0x00678D74]
	(No symbol) [0x0055C323]
	(No symbol) [0x0059DC86]
	(No symbol) [0x0059DECB]
	(No symbol) [0x005DB9F2]
	(No symbol) [0x005BFED4]
	(No symbol) [0x005D9579]
	(No symbol) [0x005BFC26]
	(No symbol) [0x0059219C]
	(No symbol) [0x0059311D]
	GetHandleVerifier [0x00998D93+2818227]
	GetHandleVerifier [0x009F542E+3196750]
	GetHandleVerifier [0x009ED9D2+3165426]
	GetHandleVerifier [0x0078DA70+675216]
	(No symbol) [0x00681B3D]
	(No symbol) [0x0067EA18]
	(No symbol) [0x0067EBB5]
	(No symbol) [0x00671640]
	BaseThreadInitThunk [0x763FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7743809E+286]
	RtlGetAppContainerNamedObjectPath [0x7743806E+238]



KeyboardInterrupt: 

# 추천순

In [4]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 음식점 리스트 파일 로드
with open('./200m아래_음식점리스트_혜정지연_합본.txt', 'r', encoding='utf-8') as file:
    store_names = [line.strip() for line in file.readlines()]

for store_name in store_names:
    try:
        # 네이버 지도 검색 URL 생성 및 이동
        search_url = f'https://map.naver.com/v5/search/{store_name}'
        driver.get(search_url)
        
        # 첫 번째 검색 결과 클릭
        try:
            first_result_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'search_title')]"))
            )
            first_result_element.click()
            print(f"'{store_name}'의 첫 번째 검색 결과 클릭 완료")
        except Exception as e:
            print(f"첫 번째 검색 결과 클릭 오류: {e}")
            continue  # 다음 가게로 넘어감

        # 페이지 로드 및 프레임 전환
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

        # 기본 정보 가져오기
        store_name_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
        )
        category_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
        )
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating = rating_element.text.replace("별점", "").strip()

        print(f"{store_name} 정보 수집 중...")

        # 최신순 리뷰 탭 클릭
        try:
            recommend_tab_element = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//*[@id='app-root']/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a"))
            )
            recommend_tab_element.click()
            time.sleep(3)
        except Exception as e:
            print(f"최신순 탭 클릭 오류: {e}")

        # 리뷰 수집
        reviews = []
        review_text_set = set()  # 중복 방지용 set
        review_count = 0

        while review_count < 1000:  # 최대 1000개 리뷰 수집
            review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
            for i, element in enumerate(review_elements, start=1):
                try:
                    # 리뷰 텍스트와 날짜 경로 지정
                    review_text_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'

                    # 리뷰 텍스트와 날짜 추출
                    review_text_element = driver.find_element(By.XPATH, review_text_xpath)
                    review_date_element = driver.find_element(By.XPATH, review_date_xpath)

                    review_text = review_text_element.text.strip()
                    review_date = review_date_element.text.strip()

                    # 중복 확인 및 저장
                    if review_text and review_text not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_text)
                        review_count += 1
                        print(f"{store_name} 리뷰 {review_count}: {review_text} ({review_date})")
                    else:
                        print(f"{store_name}: 중복 리뷰 발견, 저장하지 않음")
                except (NoSuchElementException, TimeoutException):
                    continue

            # 하단 '더보기' 버튼 클릭
            try:
                bottom_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
                )
                driver.execute_script("arguments[0].click();", bottom_more_button)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
                )
            except (NoSuchElementException, TimeoutException):
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(1)
                if len(review_text_set) >= review_count:
                    print(f"{store_name}의 리뷰 수집 완료 또는 더 이상 추가 리뷰가 없습니다.")
                    break

        # CSV 파일 저장
        reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
        reviews_df.index = range(1, len(reviews) + 1)
        csv_filename = f"{store_name}_reviews.csv"
        reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
        print(f"{store_name} 리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.\n")

    except Exception as e:
        print(f"{store_name}에서 오류 발생: {e}")

driver.quit()

첫 번째 검색 결과 클릭 오류: Message: 
Stacktrace:
	GetHandleVerifier [0x006EEBD3+24307]
	(No symbol) [0x00678D74]
	(No symbol) [0x0055C323]
	(No symbol) [0x0059DC86]
	(No symbol) [0x0059DECB]
	(No symbol) [0x005DB9F2]
	(No symbol) [0x005BFED4]
	(No symbol) [0x005D9579]
	(No symbol) [0x005BFC26]
	(No symbol) [0x0059219C]
	(No symbol) [0x0059311D]
	GetHandleVerifier [0x00998D93+2818227]
	GetHandleVerifier [0x009F542E+3196750]
	GetHandleVerifier [0x009ED9D2+3165426]
	GetHandleVerifier [0x0078DA70+675216]
	(No symbol) [0x00681B3D]
	(No symbol) [0x0067EA18]
	(No symbol) [0x0067EBB5]
	(No symbol) [0x00671640]
	BaseThreadInitThunk [0x763FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7743809E+286]
	RtlGetAppContainerNamedObjectPath [0x7743806E+238]

첫 번째 검색 결과 클릭 오류: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x006EEBD3+24307]
	(No symbol) [0x00678D74]
	(No symbol) [0x0055C323]
	(No

# 최신순